# Bad Demo: One-Shot LLM on Raw Data

## ⚠️ Anti-Pattern Demonstration

**This notebook intentionally demonstrates the WRONG way to use LLMs for analytics.**


"Which channel mix change is most likely to improve CAC next month, given a recent anomaly in referral traffic?"

In [4]:
# Setup
import os
import duckdb
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

# Load environment
load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Connect to database
db_path = '../data/synthetic_data.duckdb'
conn = duckdb.connect(db_path, read_only=True)

print("✓ Environment loaded")
print("✓ Database connected")

✓ Environment loaded
✓ Database connected


## Step 1: Raw Table Inventory

Let's show the LLM our raw tables without any semantic guidance about:
- How to join them safely
- Which metrics are canonical
- What time windows to use
- How to attribute revenue

In [5]:
# Get table schemas
tables = conn.execute("SHOW TABLES").fetchall()
print("Available tables:")
for table in tables:
    print(f"  - {table[0]}")

# Build raw schema description for LLM
schema_description = "Database Schema:\n\n"

for table in tables:
    table_name = table[0]
    columns = conn.execute(f"DESCRIBE {table_name}").fetchall()
    schema_description += f"{table_name}:\n"
    for col in columns:
        schema_description += f"  - {col[0]} ({col[1]})\n"
    schema_description += "\n"

# Show a preview
print("\n" + schema_description[:800] + "...")

Available tables:
  - dim_adgroups
  - dim_campaigns
  - dim_creatives
  - dim_customers
  - dim_products
  - fact_ad_spend
  - fact_orders
  - fact_sessions

Database Schema:

dim_adgroups:
  - adgroup_id (VARCHAR)
  - campaign_id (VARCHAR)
  - audience (VARCHAR)
  - placement (VARCHAR)

dim_campaigns:
  - campaign_id (VARCHAR)
  - channel (VARCHAR)
  - campaign_name (VARCHAR)
  - start_date (DATE)
  - end_date (DATE)
  - objective (VARCHAR)

dim_creatives:
  - creative_id (VARCHAR)
  - adgroup_id (VARCHAR)
  - format (VARCHAR)
  - asset_url (VARCHAR)

dim_customers:
  - customer_id (VARCHAR)
  - first_visit_date (DATE)
  - region (VARCHAR)
  - loyalty_segment (VARCHAR)
  - primary_device (VARCHAR)
  - acquisition_channel (VARCHAR)

dim_products:
  - sku (VARCHAR)
  - category (VARCHAR)
  - subcategory (VARCHAR)
  - brand (VARCHAR)
  - price (DOUBLE)
  - margin_pct (DOUBLE)
  - margin (DOUBLE)

fact_ad_spend:
  - date (DATE)
  - campaign_id (VA...


## Step 2: One-Shot Prompt Execution

Now we'll ask the LLM to write SQL to analyze the business question.



In [6]:
business_question = """Which channel mix change is most likely to improve CAC next month, 
given a recent anomaly in referral traffic?"""

prompt = f"""{schema_description}

Business Question: {business_question}

You have just gotten a schema description for a set of tables in our warehouse with descriptive column names.

Write a single SQL query to analyze this question and provide a recommendation.

Return your response in this format:
SQL:
[your SQL query]
"""

# Call LLM
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7
)

llm_output = response.choices[0].message.content
print("LLM Response:")
print("=" * 80)
print(llm_output)

LLM Response:
To analyze the channel mix change that is most likely to improve Customer Acquisition Cost (CAC) next month, especially considering a recent anomaly in referral traffic, we can examine the ad spend and the performance of different campaigns and channels over the past month. We will calculate the CAC for each channel and then identify which channel has the potential for improvement based on the performance metrics such as impressions, clicks, and conversions.

Here is the SQL query that addresses this analysis:

```sql
WITH channel_performance AS (
    SELECT 
        c.channel,
        SUM(fas.spend) AS total_spend,
        SUM(fas.impressions) AS total_impressions,
        SUM(fas.clicks) AS total_clicks,
        SUM(fa.pages_viewed) AS total_pages_viewed,
        SUM(CASE WHEN fa.converted_flag THEN 1 ELSE 0 END) AS total_conversions,
        SUM(fa.revenue) AS total_revenue
    FROM 
        fact_ad_spend fas
    JOIN 
        dim_campaigns c ON fas.campaign_id = c.cam

In [ ]:
# Extract and execute the SQL
import re

# Parse SQL from response
sql_match = re.search(r'SQL:(.*?)(?:Explanation:|$)', llm_output, re.DOTALL)
if sql_match:
    generated_sql = sql_match.group(1).strip()
    # Remove markdown code blocks if present
    generated_sql = re.sub(r'^```sql\s*', '', generated_sql)
    generated_sql = re.sub(r'^```\s*', '', generated_sql)
    generated_sql = re.sub(r'```\s*$', '', generated_sql)
    generated_sql = generated_sql.strip()
    
    print("\nExecuting generated SQL...\n")
    print(generated_sql)
    print("\n" + "=" * 80)
    
    try:
        result_df = conn.execute(generated_sql).df()
        print("\nQuery Results:")
        print(result_df)
        
        # Store for failure analysis
        bad_results = result_df
        bad_sql = generated_sql
    except Exception as e:
        print(f"\nQuery failed: {e}")
        bad_results = None
        bad_sql = generated_sql
else:
    print("Could not parse SQL from LLM response")
    bad_results = None
    bad_sql = None

## Step 3: Failure Exhibit

Let's analyze what went wrong with the LLM-generated SQL.

In [ ]:
# Check for specific failure patterns
failures = []

if bad_sql:
    sql_lower = bad_sql.lower()
    
    # Failure 1: Revenue attribution error
    if 'fact_orders' in sql_lower and 'fact_sessions' not in sql_lower:
        failures.append({
            'failure': 'Revenue Attribution Error',
            'description': 'Joined fact_orders directly to dim_campaigns without going through fact_sessions',
            'impact': 'Revenue is incorrectly attributed; many orders will be dropped or duplicated',
            'correct': 'Must use: fact_orders → fact_sessions → dim_campaigns (last-touch attribution)'
        })
    
    # Failure 2: Many-to-many inflation
    if 'fact_sessions' in sql_lower and 'fact_ad_spend' in sql_lower:
        if sql_lower.count('join') >= 2 and 'session_id' not in sql_lower:
            failures.append({
                'failure': 'Many-to-Many Cartesian Explosion',
                'description': 'Joined fact_sessions and fact_ad_spend on campaign_id without proper grain',
                'impact': 'Row counts multiply incorrectly; metrics are inflated by 10-100x',
                'correct': 'These tables must be aggregated separately before joining, or use semantic layer'
            })
    
    # Failure 3: Time window drift
    date_filters = re.findall(r'(date|timestamp).*?interval.*?(\d+)', sql_lower)
    if len(set([d[1] for d in date_filters])) > 1:
        failures.append({
            'failure': 'Time Window Drift',
            'description': 'Different tables use different date ranges (e.g., 30 days for spend, 90 days for revenue)',
            'impact': 'CAC calculation mixes mismatched time periods; results are meaningless',
            'correct': 'All metrics must use the same canonical window (default: 90 days)'
        })
    
    # Failure 4: Metric misuse
    if 'order_id' in sql_lower and ('cac' in sql_lower or 'acquisition' in sql_lower):
        failures.append({
            'failure': 'Metric Definition Error',
            'description': 'Used orders in CAC calculation instead of conversions',
            'impact': 'CAC is understated because not all sessions convert to orders',
            'correct': 'CAC = spend / conversions, where conversions come from fact_sessions.converted_flag'
        })
    
    # Failure 5: Dimension ambiguity
    if 'utm_source' in sql_lower or 'source' in sql_lower:
        failures.append({
            'failure': 'Dimension Ambiguity',
            'description': 'Mixed dim_campaigns.channel with utm_source or other dimension',
            'impact': 'Channels are inconsistently defined; totals don\'t match',
            'correct': 'Must use canonical dimension: dim_campaigns.channel only'
        })

print("\n" + "=" * 80)
print("FAILURE ANALYSIS")
print("=" * 80)

for i, failure in enumerate(failures, 1):
    print(f"\n{i}. {failure['failure']}")
    print(f"   What happened: {failure['description']}")
    print(f"   Impact: {failure['impact']}")
    print(f"   Correct approach: {failure['correct']}")

if not failures:
    print("\nNo obvious failures detected in pattern matching, but the results are still likely wrong!")
    print("This demonstrates another problem: it's hard to even detect when one-shot LLM SQL fails.")

## Step 4: The Overconfident Narrative

Despite these errors, the LLM produced a confident recommendation:

In [ ]:
# Extract insight
insight_match = re.search(r'INSIGHT:(.*)', llm_output, re.DOTALL)
if insight_match:
    insight = insight_match.group(1).strip()
    print("LLM Recommendation:")
    print("=" * 80)
    print(insight)
    print("=" * 80)
    print("\n⚠️ This recommendation is based on WRONG DATA due to the failures above.")
    print("⚠️ Implementing this could waste budget and harm business performance.")

## Step 5: Post-Mortem Summary

### What We Learned

**Single LLM calls on raw data fail because:**

1. **No semantic contracts**: The LLM doesn't know the canonical metric definitions
2. **No join validation**: Complex schemas allow many wrong join paths
3. **No time window enforcement**: Different metrics drift to different periods
4. **No grain management**: Many-to-many joins cause silent data explosions
5. **Overconfidence**: The LLM has no way to know it's wrong

### The Right Way

The solution requires:
- **Semantic layer**: Canonical metrics with tested SQL templates
- **Modular agents**: Small, focused, testable components
- **Deterministic logic**: Use LLMs only for ambiguity resolution
- **Observability**: Log every decision and SQL execution
- **Testing**: Validate joins, grains, and results

### Next Steps

See `02_good_modular_dspy.ipynb` for the correct implementation using:
- DSPy agent architecture
- Semantic layer from `config/semantic.yml`
- Reproducible, testable, explainable analytics

## Example 2: Vague Request with No Acceptance Criteria

Let's try another anti-pattern: asking the LLM to "analyze" data without:
- Specific questions
- Clear success criteria
- Defined scope
- Expected output format
- Constraints on what to look for

This is common in practice: analysts ask LLMs to "find something interesting" or "analyze our data".

In [ ]:
# Ask a vague, open-ended question
vague_question = """Analyze our marketing data and tell me something interesting that could help improve performance."""

vague_prompt = f"""{schema_description}

Task: {vague_question}

Look at our marketing data and provide insights. Write SQL queries as needed and explain what you found.
"""

# Call LLM with no constraints
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": vague_prompt}],
    temperature=0.7
)

vague_output = response.choices[0].message.content
print("LLM Response to Vague Request:")
print("=" * 80)
print(vague_output)

### What's Wrong with This Approach?

Even if the SQL executes, this approach fails because:

1. **No Clear Goal**: "Something interesting" is subjective - interesting to who? For what purpose?

2. **No Validation Criteria**: How do we know if the insight is:
   - Correct?
   - Actionable?
   - Novel?
   - Relevant to business goals?

3. **No Scope**: The LLM might:
   - Look at irrelevant metrics
   - Use wrong time windows
   - Mix incompatible data
   - Generate obvious or useless insights

4. **No Output Format**: The response could be:
   - A single number without context
   - A complex visualization we can't implement
   - Multiple contradictory insights
   - Generic advice that doesn't use our data

5. **Not Reproducible**: Running this again with the same data might produce completely different "insights"

6. **Not Testable**: How do you write a test for "tell me something interesting"?

In [ ]:
# Demonstrate the problems
print("\n" + "=" * 80)
print("PROBLEMS WITH VAGUE REQUESTS")
print("=" * 80)

problems = [
    {
        'problem': 'No Clear Success Criteria',
        'example': 'LLM says "Snapchat performs well" - is that interesting? Actionable? True?',
        'impact': 'Cannot validate if the insight is useful or correct'
    },
    {
        'problem': 'Generic, Obvious Insights',
        'example': 'LLM might say "Channels with lower CAC are more efficient" (duh!)',
        'impact': 'Waste time on insights that provide no value'
    },
    {
        'problem': 'Mixing Time Windows',
        'example': 'Compares last month Snapchat to last year Google Search',
        'impact': 'Invalid comparisons that mislead decision-making'
    },
    {
        'problem': 'No Prioritization',
        'example': 'LLM lists 10 "interesting" things without ranking importance',
        'impact': 'Cannot decide which insight to act on first'
    },
    {
        'problem': 'Hallucinated Patterns',
        'example': 'LLM "finds" a trend that is just random noise',
        'impact': 'False insights lead to wrong business decisions'
    }
]

for i, p in enumerate(problems, 1):
    print(f"\n{i}. {p['problem']}")
    print(f"   Example: {p['example']}")
    print(f"   Impact: {p['impact']}")

print("\n" + "=" * 80)
print("WHAT GOOD LOOKS LIKE")
print("=" * 80)

print("""
Instead of vague requests, use:

✅ SPECIFIC QUESTION:
   "Which channel mix change is most likely to improve CAC next month?"

✅ CLEAR CONSTRAINTS:
   - Time window: Last 90 days
   - Metrics: CAC, ROAS by channel
   - Output: Specific reallocation percentage
   - Success criteria: Quantified CAC improvement with confidence interval

✅ VALIDATION:
   - Check that channels mentioned exist in data
   - Verify that percentage is reasonable (3-10%)
   - Ensure output includes specific dollar amounts
   - Validate that math is correct

✅ STRUCTURED OUTPUT:
   - Hypothesis: "Shift X% from [channel] to [channel]"
   - Expected impact: "$X.XX improvement"
   - Confidence: "95% CI: [$Y, $Z]"
   - Risks: 2-3 specific risks

✅ REPRODUCIBLE:
   - Same question → same answer (or within CI)
   - Documented methodology
   - Versioned semantic definitions
   - Logged decision process

See 02_good_modular_dspy.ipynb for this approach in action.
""")

### Comparison: Vague vs Specific

| Aspect | Vague Request | Specific Request (Good Demo) |
|--------|--------------|------------------------------|
| **Question** | "Tell me something interesting" | "Which channel mix change improves CAC?" |
| **Success Criteria** | ??? | Quantified CAC improvement with CI |
| **Scope** | Entire database, any metric | CAC and ROAS by channel, 90 days |
| **Output Format** | Freeform text | Structured JSON with hypothesis |
| **Validation** | Impossible | Check against semantic catalog |
| **Actionability** | Maybe? | Clear: shift X% budget from A to B |
| **Reproducibility** | No | Yes (run_id + versioned specs) |
| **Testability** | No | Yes (inline tests) |

**Key Insight**: The specificity of the request determines the quality of the output.

Vague inputs → vague outputs, even with perfect LLMs.

Specific, constrained inputs → actionable, validated outputs.

In [ ]:
# Cleanup
conn.close()
print("\n✓ Demo complete. Database connection closed.")
print("\nRemember: This is what NOT to do. See the good demo for the right approach.")